In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlsxwriter
import pandas_flavor as pf

In [9]:
header_row = 1  # Use the second row as the column names
skip_rows = [2, 3]  # Skip the first row, and the third and fourth rows

# Read the CSV file with the specified header row and skipping the specified rows
data1 = pd.read_csv("G:\Shared drives\FMA-G\C-CONSERVACIÓN\C1_Bosque Pehuen\C1.5_Bases de datos\Estacion Meterorológica\Linea de tiempo completa\CR800Series_Table 21092018_30012020.dat", header=header_row, skiprows=skip_rows, low_memory=False)

# Print the DataFrame
print(data1)


                 TIMESTAMP  RECORD  AirTC_Max  AirTC_Avg  AirTC_Min  RH_Max  \
0      2018-09-21 16:45:00   11138      6.401      6.171      5.993  100.00   
1      2018-09-21 17:00:00   11139      6.401      6.296      6.196  100.00   
2      2018-09-21 17:15:00   11140      6.469      6.321      6.230  100.00   
3      2018-09-21 17:30:00   11141      6.537      6.465      6.401  100.00   
4      2018-09-21 17:45:00   11142      6.674      6.554      6.401  100.00   
...                    ...     ...        ...        ...        ...     ...   
47595  2020-01-30 11:30:00   58733     15.860     15.530     15.250   54.63   
47596  2020-01-30 11:45:00   58734     16.200     15.910     15.720   51.77   
47597  2020-01-30 12:00:00   58735     16.950     16.570     16.200   44.81   
47598  2020-01-30 12:15:00   58736     16.880     16.700     16.330   48.22   
47599  2020-01-30 12:30:00   58737     17.070     16.860     16.610   47.04   

       RH_Avg  RH_Min  WS_ms_Max  WS_ms_Avg  ...  D

In [10]:
print(data1.columns)

Index(['TIMESTAMP', 'RECORD', 'AirTC_Max', 'AirTC_Avg', 'AirTC_Min', 'RH_Max',
       'RH_Avg', 'RH_Min', 'WS_ms_Max', 'WS_ms_Avg', 'WS_ms_Min',
       'WindDir_Max', 'WindDir_Avg', 'WindDir_Min', 'WindDir_Std',
       'BP_mbar_Avg', 'T107_10cm_Max', 'T107_10cm_Avg', 'T107_10cm_Min',
       'T107_10cm_Std', 'T107_50cm_Max', 'T107_50cm_Avg', 'T107_50cm_Min',
       'T107_50cm_Std', 'Rain_mm_Tot', 'PTemp_C_Avg', 'PtoRocio_Avg',
       'BattV_Min', 'DT_Max', 'DT_Avg', 'DT_Min', 'Q_Max', 'Q_Min', 'TCDT_Max',
       'TCDT_Min', 'incomingLW_Avg', 'incomingSW_Avg', 'outgoingLW_Avg',
       'outgoingSW_Avg', 'albedo_Avg'],
      dtype='object')


In [12]:
duplicates = data1['RECORD'].duplicated() 
print(any(duplicates))

False
